In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory


Using TensorFlow backend.


In [2]:
data = pd.read_csv('../input/Sentiment.csv')
data = data[['text','sentiment']]

In [3]:
data = data[data.sentiment != "Neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['sentiment'] == 'Positive'].size)
print(data[ data['sentiment'] == 'Negative'].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
batch_size = 17500
tokenizer = Tokenizer(nb_words=batch_size, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)


4472
16986


In [4]:
embed_dim = 128
lstm_out = 128
model = Sequential()
model.add(Embedding(batch_size, embed_dim,input_length = X.shape[1], dropout=0.2))
model.add(LSTM(lstm_out, dropout_U=0.2, dropout_W=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['fbeta_score'])
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (None, 29, 128)       2240000     embedding_input_1[0][0]          
____________________________________________________________________________________________________
lstm_1 (LSTM)                    (None, 128)           131584      embedding_1[0][0]                
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 2)             258         lstm_1[0][0]                     
Total params: 2,371,842
Trainable params: 2,371,842
Non-trainable params: 0
____________________________________________________________________________________________________
None


In [5]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(7188, 29) (7188, 2)
(3541, 29) (3541, 2)


In [6]:
model.fit(X_train, Y_train, nb_epoch = 15, batch_size=32, verbose = 2)

/opt/conda/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:91: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/15
71s - loss: 0.4536 - fbeta_score: 0.8082
Epoch 2/15
69s - loss: 0.2972 - fbeta_score: 0.8758
Epoch 3/15
67s - loss: 0.2005 - fbeta_score: 0.9217
Epoch 4/15
69s - loss: 0.1458 - fbeta_score: 0.9445
Epoch 5/15
73s - loss: 0.1241 - fbeta_score: 0.9514
Epoch 6/15
73s - loss: 0.1119 - fbeta_score: 0.9573
Epoch 7/15
72s - loss: 0.0980 - fbeta_score: 0.9620
Epoch 8/15
72s - loss: 0.0936 - fbeta_score: 0.9608
Epoch 9/15
71s - loss: 0.0826 - fbeta_score: 0.9659
Epoch 10/15
73s - loss: 0.0883 - fbeta_score: 0.9634
Epoch 11/15
72s - loss: 0.0847 - fbeta_score: 0.9634
Epoch 12/15
72s - loss: 0.0812 - fbeta_score: 0.9662
Epoch 13/15
71s - loss: 0.0765 - fbeta_score: 0.9684
Epoch 14/15
71s - loss: 0.0793 - fbeta_score: 0.9684
Epoch 15/15
69s - loss: 0.0770 - fbeta_score: 0.9673


In [7]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score = model.evaluate(X_test, Y_test, verbose = 2)
print("score: %.2f" % (score[1]))

score: 0.84


In [8]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1

    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1

print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

pos_acc 57.60517799352751 %
neg_acc 91.93954659949623 %
